In [2]:
%pylab inline
import os
import numpy as np
import pandas as pd
from scipy.misc import imread
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
import re

import tensorflow as tf
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Convolution2D, Flatten, MaxPooling2D, Reshape, InputLayer
from keras.optimizers import SGD

root_dir = os.path.abspath('/Users/dcolton/Desktop/checked/')
os.path.exists(root_dir)

seed = 128
rng = np.random.RandomState(seed)

Populating the interactive namespace from numpy and matplotlib


/Users/dcolton/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [3]:
train = pd.read_csv(os.path.join(root_dir, 'train.csv'))
sample_submission = pd.read_csv(os.path.join(root_dir, 'train.csv'))

In [4]:
dfpngdf = pd.read_csv('checked/pngdf1.csv', sep='\t', header=None)
dfb = (pd.DataFrame(dfpngdf[0].map(lambda x: x[:-4])))
df = pd.concat([dfpngdf, dfb], axis = 1)
df.columns = ['png','Title']

In [20]:
import os
d='./capstone/Checked/'
[os.path.join(d,o) for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]

['./capstone/Checked/A : Fsm',
 './capstone/Checked/Ab : Fm',
 './capstone/Checked/B : Cb : Gsm',
 './capstone/Checked/Bb : Gm',
 './capstone/Checked/C : Am',
 './capstone/Checked/D : Bm',
 './capstone/Checked/Db : Cs : Bbm',
 './capstone/Checked/E : Csm',
 './capstone/Checked/Eb : Cm',
 './capstone/Checked/F : Dm',
 './capstone/Checked/Fs : Gb : Ebm',
 './capstone/Checked/G : Em']

In [33]:
import sys,os

root = "/Users/dcolton/Desktop/capstone/Checked/"
path = os.path.join(root, "targetdirectory")

i = 0
dff = pd.DataFrame(columns = ['path', 'name'])

dff.columns = ['path', 'name']

for path, subdirs, files in os.walk(root):
    for name in files:
        if name.endswith(".png"):
#             print os.path.join(path) + '!!!' + os.path.join(name)
#             dff.append(os.path.join(path), os.path.join(name))
            i += 1
            
print dff
            
#         print os.path.join(path, name)


# for file in os.listdir("/mydir"):
#     if file.endswith(".txt"):
#         print(os.path.join("/mydir", file))


ValueError: Length mismatch: Expected axis has 0 elements, new values have 2 elements

In [5]:
namecol = []

for t in df.Title:
    i = t[-3:]
    m = re.match(".+(-\d{1,2})",i)
    if m:
        t = t.replace(m.group(1),"")
    namecol.append(t)
print len(namecol)

3232


In [6]:
df['Stripped']=np.array(namecol)
df['exists'] = np.array(-1)

In [7]:
train['Stripped']=train['Title']

In [8]:
dfnew = pd.merge(df, train, how='left',on='Stripped')

In [9]:
unmatched = []
for row in train.values:
    title = row[12]
    label = row[9]
    df.exists[df.Stripped==title] = label
    if sum(df.Stripped == title)==0:
        unmatched.append([title, row])
    
    

/Users/dcolton/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
df[df['exists']==-1]

,png,Title,Stripped,exists
34,"Ach, ich fühl's, es ist verschwunden The Magi...","Ach, ich fühl's, es ist verschwunden The Magi...","Ach, ich fühl's, es ist verschwunden The Magi...",-1
35,"Ach, ich fühl's, es ist verschwunden The Magi...","Ach, ich fühl's, es ist verschwunden The Magi...","Ach, ich fühl's, es ist verschwunden The Magi...",-1
54,"Adagio Flute Sonata No. 6 in A minor, “La Boug...","Adagio Flute Sonata No. 6 in A minor, “La Boug...","Adagio Flute Sonata No. 6 in A minor, “La Bouget”",-1
55,"Adagio Flute Sonata No. 6 in A minor, “La Boug...","Adagio Flute Sonata No. 6 in A minor, “La Boug...","Adagio Flute Sonata No. 6 in A minor, “La Bouget”",-1
113,"Adagio cantabile Piano Sonata No. 8, “Pathéti...","Adagio cantabile Piano Sonata No. 8, “Pathéti...","Adagio cantabile Piano Sonata No. 8, “Pathéti...",-1
114,"Adagio cantabile Piano Sonata No. 8, “Pathéti...","Adagio cantabile Piano Sonata No. 8, “Pathéti...","Adagio cantabile Piano Sonata No. 8, “Pathéti...",-1
115,"Adagio cantabile Piano Sonata No. 8, “Pathéti...","Adagio cantabile Piano Sonata No. 8, “Pathéti...","Adagio cantabile Piano Sonata No. 8, “Pathéti...",-1
116,"Adagio cantabile Piano Sonata No. 8, “Pathéti...","Adagio cantabile Piano Sonata No. 8, “Pathéti...","Adagio cantabile Piano Sonata No. 8, “Pathéti...",-1
117,"Adagio cantabile Piano Sonata No. 8, “Pathéti...","Adagio cantabile Piano Sonata No. 8, “Pathéti...","Adagio cantabile Piano Sonata No. 8, “Pathéti...",-1
136,Air à l'Italien Suite in A minor for Recorder...,Air à l'Italien Suite in A minor for Recorder...,Air à l'Italien Suite in A minor for Recorder...,-1


In [11]:
asdf = df['exists'].value_counts()

In [12]:
dfe = df[df['exists']!=-1]

In [13]:
dfe['exists'].value_counts()

1     773
3     683
8     393
4     263
12    220
11    207
10    157
6      63
2       8
Name: exists, dtype: int64

In [14]:
train[train['Label']==2]

,Unnamed: 0,Difficulty_Level,Key,Link,Range,Tempo,Time_Signature,Title,Key.1,Label,Unnamed: 10,filename,Stripped
2202,778,easy,E minor,/tunes/handel-flute-sonata-in-g-major-iii-adag...,"('F#', '4', 'B', '5')",69 BPM,4-Mar,Adagio Flute Sonata in G major,Em,2,/tunes/handel-flute-sonata-in-g-major-iii-adagio,handel-flute-sonata-in-g-major-iii-adagio,Adagio Flute Sonata in G major
2203,2063,easy,E minor,/tunes/marcello-recorder-sonata-in-e-minor-op2...,"('G', '4', 'C', '6')",56 BPM,4-Apr,Adagio Recorder Sonata No. 4 in E minor,Em,2,/tunes/marcello-recorder-sonata-in-e-minor-op2...,marcello-recorder-sonata-in-e-minor-op2-no4-ii...,Adagio Recorder Sonata No. 4 in E minor
2204,1834,intermediate,E minor,/tunes/albinoni-trattenimenti-armonici-no9-iii...,"('E', '4', 'B', '5')",56 BPM,4-Mar,"Adagio Trattenimenti armonici, Sonata No. 9 in...",Em,2,/tunes/albinoni-trattenimenti-armonici-no9-iii...,albinoni-trattenimenti-armonici-no9-iii-adagio,"Adagio Trattenimenti armonici, Sonata No. 9 in..."
2205,1335,easy,E minor,/tunes/handel-flute-sonata-1b-in-e-minor-adagi...,"('G', '4', 'B', '5')",60 BPM,4-Mar,Adagio Flute Sonata No. 1b in E minor,Em,2,/tunes/handel-flute-sonata-1b-in-e-minor-adagio,handel-flute-sonata-1b-in-e-minor-adagio,Adagio Flute Sonata No. 1b in E minor
2206,2127,easy,E minor,/tunes/marcello-recorder-sonata-in-g-major-op2...,"('B', '4', 'C', '6')",69 BPM,4-Mar,Adagio Recorder Sonata No. 5 in G major,Em,2,/tunes/marcello-recorder-sonata-in-g-major-op2...,marcello-recorder-sonata-in-g-major-op2-no5-ii...,Adagio Recorder Sonata No. 5 in G major
2207,1360,intermediate,E minor,/tunes/handel-flute-sonata-hwv375-in-e-minor-a...,"('G', '4', 'C', '6')",46 BPM,4-Apr,Adagio Flute Sonata in E minor,Em,2,/tunes/handel-flute-sonata-hwv375-in-e-minor-a...,handel-flute-sonata-hwv375-in-e-minor-adagio,Adagio Flute Sonata in E minor
2208,1630,easy,E minor,/tunes/loeillet-de-gant-op5-no6-ii-adagio.pdf,"('G', '4', 'D', '6')",60 BPM,4-Mar,Adagio Sonata for Two Flutes No. 6 in E minor,Em,2,/tunes/loeillet-de-gant-op5-no6-ii-adagio,loeillet-de-gant-op5-no6-ii-adagio,Adagio Sonata for Two Flutes No. 6 in E minor
2209,1662,intermediate,E minor,/tunes/blavet-op1-sonata-no1-i-adagio.pdf,"('D', '4', 'B', '5')",56 BPM,4-Apr,Adagio Sonata for Two Flutes No. 1 in E minor,Em,2,/tunes/blavet-op1-sonata-no1-i-adagio,blavet-op1-sonata-no1-i-adagio,Adagio Sonata for Two Flutes No. 1 in E minor
2210,2230,intermediate,E minor,/tunes/blavet-op2-sonata-no3-adagio.pdf,"('E', '4', 'C', '6')",48 BPM,4-Apr,"Adagio Flute Sonata No. 3 in E minor, ���La Dh...",Em,2,/tunes/blavet-op2-sonata-no3-adagio,blavet-op2-sonata-no3-adagio,"Adagio Flute Sonata No. 3 in E minor, ���La Dh..."
2211,729,intermediate,E minor,/tunes/telemann-canonic-sonata-no1-adagio.pdf,"('D#', '4', 'B', '5')",30 BPM,4-Feb,"Adagio Six Canonic Sonatas for Two Flutes, No. 1",Em,2,/tunes/telemann-canonic-sonata-no1-adagio,telemann-canonic-sonata-no1-adagio,"Adagio Six Canonic Sonatas for Two Flutes, No. 1"


In [15]:
# dfnew['label'] = dfnew['Unnamed: 9']
# del dfnew['Unnamed: 9']
df['exists'].value_counts()

 1     773
 3     683
-1     465
 8     393
 4     263
 12    220
 11    207
 10    157
 6      63
 2       8
Name: exists, dtype: int64

In [16]:
print df.shape
print train.shape
print dfnew.shape

(3232, 4)
(3474, 13)
(3921, 16)


In [17]:
null_data = dfnew[dfnew.isnull().any(axis=1)]

In [18]:
dfnew.drop(null_data.index, inplace=True)

In [41]:
temp = []
i = 0
for img_name in dfe['png']:
    for i in xrange(15):
        if os.path.isfile('/Users/dcolton/Desktop/checked/'+ img_name + '-' + str(i)):
            image_path = os.path.join('/Users/dcolton/Desktop/checked',img_name)
            img = imread(image_path, mode='L')
            img = img.astype('float32')
            temp.append(img)
            i += 1
    if os.path.isfile('/Users/dcolton/Desktop/checked/'+ img_name):
        image_path = os.path.join('/Users/dcolton/Desktop/checked',img_name)
        img = imread(image_path, mode='L')
        img = img.astype('float32')
        temp.append(img)

In [20]:
dfe['png'].shape

(2767,)

In [ ]:
dfe['exists']

In [22]:
y = dfe['exists'].values
dfe['exists'].value_counts()

1     773
3     683
8     393
4     263
12    220
11    207
10    157
6      63
2       8
Name: exists, dtype: int64

In [139]:
x=np.stack(temp)
x = x.reshape(2767, 300, 300, 1).astype('Float32')
x /= 255.0

In [141]:
xtr, xte, ytr, yte = train_test_split(x, y, stratify=y)

In [143]:
ytr = keras.utils.np_utils.to_categorical(ytr, 13)
yte = keras.utils.np_utils.to_categorical(yte, 13)

In [92]:
ytr.shape

(2075, 13)

In [163]:
input_num_units = 90000
hidden_num_units = 20
output_num_units = 12
epochs = 20
batch_size = 128
pixel_count = 90000
language_count = 13

In [167]:
model = Sequential()
model.add(Convolution2D(16, kernel_size = (16, 16), input_shape=(300, 300, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(language_count))
model.add(Activation('softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,  metrics=['accuracy'])


# input_reshape = (300,300,1)
# pool_size = (2, 2)
# train_x_temp = xtr.reshape(-1, 300, 300, 1)
# val_x_temp = xte.reshape(-1, 300, 300, 1)

# trained_model_conv = model.fit(train_x_temp, ytr)
# #         nb_epoch=epochs, batch_size=batch_size, validation_data=(val_x_temp, yte))

In [177]:
model = Sequential()

model.add(Convolution2D(32, (3, 3), padding='same',
                 input_shape=xtr.shape[1:]))
model.add(Activation('relu'))
model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# model.add(Convolution2D(64, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(Convolution2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(language_count))
model.add(Activation('softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd,  metrics=['accuracy'])

In [ ]:
trained_model = model.fit(xtr, ytr, epochs=epochs, batch_size=batch_size, validation_data=(xte, yte))

Train on 2075 samples, validate on 692 samples
Epoch 1/20
 384/2075 [====>.........................] - ETA: 863s - loss: 4.1342 - acc: 0.1510Traceback (most recent call last):
  File "/Users/dcolton/anaconda/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1118, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/Users/dcolton/anaconda/lib/python2.7/site-packages/IPython/core/ultratb.py", line 300, in wrapped
    return f(*args, **kwargs)
  File "/Users/dcolton/anaconda/lib/python2.7/site-packages/IPython/core/ultratb.py", line 345, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/Users/dcolton/anaconda/lib/python2.7/inspect.py", line 1048, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/Users/dcolton/anaconda/lib/python2.7/inspect.py", line 1008, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  Fi

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



IndexError: string index out of range